In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-02-28 11:49:16--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  79.8MB/s    in 0.3s    

2025-02-28 11:49:16 (79.8 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [20]:
# Clean data - remove users whith less than 200 ratings and books with less than 100
df_filtered_ratings = df_ratings[df_ratings.groupby('user')['rating'].transform('count') >= 200]
df_filtered_ratings = df_filtered_ratings[df_filtered_ratings.groupby('isbn')['rating'].transform('count') >= 100]
df_filtered_ratings




,user,isbn,rating
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1495,277427,0142001740,0.0
1513,277427,0312966091,0.0
...,...,...,...
1146823,275970,0440221471,0.0
1146824,275970,0440222656,0.0
1146825,275970,0440226430,0.0
1146852,275970,044651652X,6.0


In [21]:
# Create new dataframe
# rating_count = df_filtered_ratings.groupby('isbn')['rating'].count()
# rating_avg = df_filtered_ratings.groupby('isbn')['rating'].mean()
# new_ratings_df = pd.DataFrame({'ratings_count':rating_count, 'rating_avg': rating_avg})
# df = df_books.merge(new_ratings_df, on='isbn')
# df

In [9]:
# Drop duplicates
df.drop_duplicates('title')


,isbn,title,author,ratings_count,rating_avg
0,0440234743,The Testament,John Grisham,124,1.435484
1,0971880107,Wild Animus,Rich Shapero,365,0.435616
2,0446310786,To Kill a Mockingbird,Harper Lee,139,3.863309
3,0440225701,The Street Lawyer,JOHN GRISHAM,138,1.746377
4,0804106304,The Joy Luck Club,Amy Tan,172,1.848837
...,...,...,...,...,...
94,0671001795,Two for the Dough,Janet Evanovich,126,2.182540
95,0312983271,Full House (Janet Evanovich's Full Series),Janet Evanovich,103,2.126214
96,0440222656,The Horse Whisperer,Nicholas Evans,183,1.273224
97,0553280341,B Is for Burglar (Kinsey Millhone Mysteries (P...,Sue Grafton,128,2.234375


In [19]:
# Prepare features
x_matrix = csr_matrix(df_filtered_ratings('user', 'isbn'))
print(x_matrix)

# Scale the features to avoid biasing KNN due to different scales
# scaler = StandardScaler()
# x_scaled = scaler.fit_transform(x)
# print(x_scaled)



TypeError: 'DataFrame' object is not callable

In [14]:
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(x_scaled)
knn.kneighbors([[-0.34149976, -1.02589512]])

(array([[5.51646539e-09, 1.26721839e-01, 1.82892325e-01, 2.12458397e-01,
         2.19760092e-01]]),
 array([[ 0, 24, 16, 69, 65]]))

In [17]:
# function to return recommended books - this will be tested
def get_recommends(book_title):
  book_index = df[df['title'] == book_title].index

  distances, indices = knn.kneighbors(x_scaled[book_index])

  # Get book titles
  recommended_books = df.iloc[indices[0]]['title']


  return recommended_books.tolist()

get_recommends("The Testament")


['The Testament',
 'Along Came a Spider (Alex Cross Novels)',
 'Hannibal',
 'A Map of the World',
 "The Book of Ruth (Oprah's Book Club (Paperback))"]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

['Interview with the Vampire', 'Angels &amp; Demons', 'Jurassic Park', "The Pilot's Wife : A Novel"]


IndexError: string index out of range